In [7]:
import pandas as pd
import os
import numpy as np

folder_path = 'C:\\Users\\mikke\\OneDrive - Syddansk Universitet\\Data Science\\13. Visualisering\\project\\visualisering\\data'

df_main = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_parts = filename.split('_')
        kommune_name = file_parts[1]
        animal_name = file_parts[2].split('.')[0]

        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep=';', names=['Year', 'Taken game'], header=0)

        df['Kommune'] = kommune_name
        df['Species'] = animal_name

        df_main = df_main.append(df, ignore_index=True)
        df_main = df_main[df_main['Year'] >= 2006]

C:\Users\mikke\AppData\Local\Temp\ipykernel_16116\3522365501.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_main = df_main.append(df, ignore_index=True)
C:\Users\mikke\AppData\Local\Temp\ipykernel_16116\3522365501.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_main = df_main.append(df, ignore_index=True)
C:\Users\mikke\AppData\Local\Temp\ipykernel_16116\3522365501.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_main = df_main.append(df, ignore_index=True)
C:\Users\mikke\AppData\Local\Temp\ipykernel_16116\3522365501.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_main = df_main.append(df, ignore_index=Tr

In [8]:
df_main['Species'] = df_main['Species'].str.replace("\s*\(\d+\)", "", regex=True)
df_main = df_main[df_main['Species'] != "Alle arter"]
df_main['Species'] = df_main['Species'].replace('Måger', 'Måge')
df_main['Species'] = df_main['Species'].replace('Vildkanin', 'Kanin')
df_main['Species'] = df_main['Species'].replace('Gæs', 'Gås')

In [9]:
Andefugle = ['Andre dykænder', 'Andre svømmeænder', 'Atlingand', 'Bjergand', 'Fløjlsand', 'Gråand', 'Hvinand', 'Knarand', 'Krikand',
            'Pibeand', 'Skeand', 'Sortand', 'Spidsand', 'Taffeland', 'Troldand', 'Ederfugl', 'Havlit', 'Stor skallesluger', 'Toppet skallesluger']
Gæs = ['Blisgås', 'Bramgås', 'Canadagås', 'Gås', 'Kortnæbbet gås', 'Nilgås', 'Grågås', 'Sædgås']
Hønsefugle = ['Agerhøne', 'Fasan', 'Blishøne']
Vadefugle = ['Bekkasiner', 'Dobbeltbekkasin', 'Skovsneppe']
Måger = ['Måge', 'Sildemåge', 'Sølvmåge', 'Svartbag']
Duer = ['Ringdue', 'Tyrkerdue']
Kragefugle = ['Indisk huskrage','Krage', 'Råge', 'Husskade']
Hovdyr = ['Dådyr', 'Krondyr', 'Muflon', 'Rådyr', 'Sika', 'Vildsvin']
Rovdyr = ['Ræv', 'Mårhund']
Støttetandede = ['Hare', 'Kanin']

delete = ['Amerikansk skarveand', 'Muntjak', 'Bisamrotte', 'Gråt egern', 'Ræveegern', 'Næsebjørn', 'Rødbuget egern', 'Sort svane', 'Sumpbæver', 'Vaskebjørn', 'Mink', 'Hellig ibis', 'Ilder', 'Skarv', 'Fiskehejre', 'Husmår', 'Spættet sæl', 'Stær']
df_main = df_main[~df_main['Species'].isin(delete)]

In [10]:
species_groups = {}
for group, species_list in [('Andefugle', Andefugle), ('Gæs', Gæs), 
                            ('Hønsefugle', Hønsefugle), ('Vadefugle', Vadefugle), ('Måger', Måger), 
                            ('Duer', Duer), ('Kragefugle', Kragefugle), ('Hovdyr', Hovdyr), 
                            ('Rovdyr', Rovdyr), ('Støttetandede', Støttetandede)]:
    for species in species_list:
        species_groups[species] = group

df_main['Group'] = df_main['Species'].map(species_groups)

In [11]:
df_main_species = df_main.groupby(['Year', 'Kommune', 'Species', 'Group']).agg({'Taken game': 'sum'}).reset_index()
df_main_group = df_main_species.groupby(['Year', 'Kommune', 'Group'])['Taken game'].sum().reset_index()
df_main_group = df_main_group.rename(columns={'Taken game': 'Taken game group'})
df_merged = pd.merge(df_main_species, df_main_group, on=['Year', 'Kommune', 'Group'])
df_merged.sort_values(by=['Kommune', "Year", "Species"])
df_merged.reset_index(drop=True)

,Year,Kommune,Species,Group,Taken game,Taken game group
0,2006,Aabenraa,Agerhøne,Hønsefugle,731,3707
1,2006,Aabenraa,Blishøne,Hønsefugle,49,3707
2,2006,Aabenraa,Fasan,Hønsefugle,2927,3707
3,2006,Aabenraa,Andre dykænder,Andefugle,223,8461
4,2006,Aabenraa,Andre svømmeænder,Andefugle,542,8461
...,...,...,...,...,...,...
79395,2022,Ærø,Råge,Kragefugle,113,226
79396,2022,Ærø,Mårhund,Rovdyr,0,40
79397,2022,Ærø,Ræv,Rovdyr,40,40
79398,2022,Ærø,Ringdue,Duer,28,28


Create a new csv file:

In [12]:
output_directory = 'C:/Users/mikke/OneDrive - Syddansk Universitet\Data Science/13. Visualisering/project/visualisering/'
output_filename = 'taken_game.csv'
output_path = os.path.join(output_directory, output_filename)
df_merged.to_csv(output_path, index=False)